#  OOP: Overriding 


##  Table of Contents 
* [Python Data Model](#python_data_model)
* [The Internals of Operations Like `len()` and `[]`](#internals_of_operations_like_len)
* [Overloading Built-in Functions](#overloading_built-in_functions)
    * [Giving a Length to Your Objects Using `len()`](#giving_a_length_to_your_objects_using_`len()`)
    * [Making Your Objects Work With `abs()`](#making_your_objects_work_with_`abs()`)
    * [Printing Your Objects Prettily Using `str()`](#printing_your_objects_prettily_using_`str()`)
    * [Representing Your Objects Using `repr()`](#representing_your_objects_using_`repr()`)
    * [Making Your Objects Truthy or Falsey Using `bool()`](#making_your_objects_truthy_or_falsey_using_`bool()`)
* [Overriding Built-in Operators](#overriding_built-in_operators)
    * [Making Your Objects Capable of Being Added Using `+`](#making_your_objects_capable_of_being_added_using_`+`)
    * [Indexing and Slicing Your Objects Using `[]`](#indexing_and_slicing_your_objects_using_`[]`)

---

If you’ve used the `+` or `*` operator on a `str` object in Python, you must have noticed its different behavior when compared to `int` or `float` objects:

In [ ]:
"str_1" + "str_2"

In [ ]:
"str_1" * 3

In [ ]:
2 + 3

In [ ]:
2 * 3

You might have wondered how the same built-in operator or function shows different behavior for objects of different classes. This is called operator overriding or function overriding, respectively.

<a class="anchor" id="python_data_model"></a>
## Python Data Model

Say you have a class representing an online order having a cart (a list) and a customer (a str or instance of another class which represents a customer).

In such a case, it is quite natural to want to obtain the length of the cart list. Someone new to Python might decide to implement a method called `get_cart_len()` in their class to do this. But you can configure the built-in `len()` in such a way that it returns the length of the cart list when given our object.

In another case, we might want to append something to the cart. Again, someone new to Python would think of implementing a method called `append_to_cart()` that takes an item and appends it to the cart list. But you can configure the `+` operator in such a way that it appends a new item to the cart.

Python does all this using special methods. These special methods have a naming convention, where the name starts with two underscores, followed by an identifier and ends with another pair of underscores.

Essentially, each built-in function or operator has a special method corresponding to it. For example, there’s `__len__()`, corresponding to `len()`, and `__add__()`, corresponding to the `+` operator.

By default, most of the built-ins and operators will not work with objects of your classes. You must add the corresponding special methods in your class definition to make your object compatible with built-ins and operators. When you do this, the behavior of the function or operator associated with it changes according to that defined in the method.

This is exactly what the [Data Model (Section 3 of the Python documentation)](https://docs.python.org/3/reference/datamodel.html) helps you accomplish. It lists all the special methods available and provides you with the means of overriding built-in functions and operators so that you can use them on your own objects.

> **Note:** Due to the naming convention used for these methods, they are also called **dunder methods** which is a shorthand for **d**ouble **under**score methods. Sometimes they’re also referred to as special methods or magic methods. We prefer dunder methods though!

<a class="anchor" id="internals_of_operations_like_len"></a>
## The Internals of Operations Like `len()` and `[]`

Every class in Python defines its own behavior for built-in functions and methods. When you pass an instance of some class to a built-in function or use an operator on the instance, it is actually equivalent to calling a special method with relevant arguments.

If there is a built-in function, `func()`, and the corresponding special method for the function is `__func__()`, Python interprets a call to the function as `obj.__func__()`, where obj is the object. In the case of operators, if you have an operator opr and the corresponding special method for it is `__opr__()`, Python interprets something like `obj1 <opr> obj2` as `obj1.__opr__(obj2)`.

So, when you’re calling `len()` on an object, Python handles the call as `obj.__len__()`. When you use the `[]` operator on an iterable to obtain the value at an index, Python handles it as `itr.__getitem__(index)`, where `itr` is the iterable object and index is the index you want to obtain.

Therefore, when you define these special methods in your own class, you override the behavior of the function or operator associated with them because, behind the scenes, Python is calling your method. Let’s get a better understanding of this:

In [ ]:
a = 'Real Python'
b = ['Real', 'Python']

In [ ]:
len(a)

In [ ]:
a.__len__()

In [ ]:
b[0]

In [ ]:
b.__getitem__(0)

As you can see, when you use the function or its corresponding special method, you get the same result. In fact, when you obtain the list of attributes and methods of a str object using `dir()`, you’ll see these special methods in the list in addition to the usual methods available on `str` objects:

In [ ]:
dir(a)

If the behavior of a built-in function or operator is not defined in the class by the special method, then you will get a `TypeError`.

So, how can you use special methods in your classes?

<a class="anchor" id="overloading_built-in_functions"></a>
## Overloading Built-in Functions

Many of the special methods defined in the Data Model can be used to change the behavior of functions such as `len`, `abs`, `hash`, `divmod`, and so on. To do this, you only need to define the corresponding special method in your class. Let’s look at a few examples:

<a class="anchor" id="giving_a_length_to_your_objects_using_`len()`"></a>
### Giving a Length to Your Objects Using `len()`

To change the behavior of `len()`, you need to define the `__len__()` special method in your class. Whenever you pass an object of your class to `len()`, your custom definition of `__len__()` will be used to obtain the result. Let’s implement `len()` for the order class we talked about in the beginning:

In [ ]:
class Order:
    def __init__(self, cart, customer):
        self.cart = list(cart)
        self.customer = customer
        
    def __len__(self):
        return len(self.cart)

In [ ]:
order = Order(['banana', 'apple', 'mango'], 'Ali Hejazizo')
len(order)

As you can see, you can now use `len()` to directly obtain the length of the cart. Moreover, it makes more intuitive sense to say “length of order” rather than calling something like `order.get_cart_len()`. Your call is both Pythonic and more intuitive. When you don’t have the `__len__()` method defined but still call `len()` on your object, you get a `TypeError`:

In [ ]:
class Order:
    def __init__(self, cart, customer):
        self.cart = list(cart)
        self.customer = customer

order = Order(['banana', 'apple', 'mango'], 'Real Python')
len(order)  # Calling len when no __len__

But, when overriding `len()`, you should keep in mind that Python requires the function to return an integer. If your method were to return anything other than an integer, you would get a `TypeError`. This, most probably, is to keep it consistent with the fact that `len()` is generally used to obtain the length of a sequence, which can only be an integer:

In [ ]:
class Order:
    def __init__(self, cart, customer):
        self.cart = list(cart)
        self.customer = customer

    def __len__(self):
        return float(len(self.cart))  # Return type changed to float

order = Order(['banana', 'apple', 'mango'], 'Real Python')
len(order)

In [ ]:
class Order:
    def __init__(self, cart, customer):
        self.cart = list(cart)
        self.customer = customer
        
    def __len__(self):
        return len(self.cart)

In [ ]:
order = Order(['banana', 'apple', 'mango'], 'Real Python')
len(order)

<a class="anchor" id="making_your_objects_work_with_`abs()`"></a>
### Making Your Objects Work With `abs()`

You can dictate the behavior of the abs() built-in for instances of your class by defining the `__abs__()` special method in the class.

In [ ]:
class Vector:
    def __init__(self, x_comp, y_comp):
        self.x_comp = x_comp
        self.y_comp = y_comp

    def __abs__(self):
        return (self.x_comp ** 2 + self.y_comp ** 2) ** 0.5

In [ ]:
vector = Vector(3, 4)
abs(vector)

It makes more intuitive sense to say “absolute value of vector” rather than calling something like `vector.get_mag()`.

<a class="anchor" id="printing_your_objects_prettily_using_`str()`"></a>
### Printing Your Objects Prettily Using `str()`

The `str()` built-in is used to cast an instance of a class to a str object, or more appropriately, to obtain a user-friendly string representation of the object which can be read by a normal user rather than the programmer. You can define the string format your object should be displayed in when passed to `str()` by defining the `__str__()` method in your class. Moreover, `__str__()` is the method that is used by Python when you call `print()` on your object.

In [ ]:
class Vector:
    def __init__(self, x_comp, y_comp):
        self.x_comp = x_comp
        self.y_comp = y_comp

    def __str__(self):
        # By default, sign of +ve number is not displayed
        # Using `+`, sign is always displayed
        return f'{self.x_comp}i + {self.y_comp}j'

In [ ]:
vector = Vector(3, 4)
str(vector)

In [ ]:
print(vector)

<a class="anchor" id="representing_your_objects_using_`repr()`"></a>
### Representing Your Objects Using `repr()`

Read [Python String Conversion 101: Why Every Class Needs a “repr”](https://dbader.org/blog/python-repr-vs-str) for a complete tutorial on `__repr__` and the difference between `__repr__` and `__str__`.

[“Difference between `__str__` and `__repr__` in Python”](https://stackoverflow.com/questions/1436703/what-is-the-difference-between-str-and-repr) is a Stack Overflow question with excellent contributions from Pythonistas Alex Martelli and Martijn Pieters.

Rule of thumb: `__repr__` is for developers, `__str__` is for customers.

<a class="anchor" id="making_your_objects_truthy_or_falsey_using_`bool()`"></a>
### Making Your Objects Truthy or Falsey Using `bool()`

The `bool()` built-in can be used to obtain the truth value of an object. To define its behavior, you can use the `__bool__()`.

The behavior defined here will determine the truth value of an instance in all contexts that require obtaining a truth value such as in `if` statements.

In [ ]:
class Order:
    def __init__(self, cart, customer):
        self.cart = list(cart)
        self.customer = customer

    def __bool__(self):
        return len(self.cart) > 0

In [ ]:
order1 = Order(['banana', 'apple', 'mango'], 'Ali Hejazizo')
order2 = Order([], 'Mohsen')

bool(order1), bool(order2)

In [ ]:
for order in [order1, order2]:
    if order:
        print(f"{order.customer}'s order is processing...")
    else:
        print(f"Empty order for customer {order.customer}")

> **Note:** When the `__bool__()` special method is not implemented in a class, the value returned by `__len__()` is used as the truth value, where a non-zero value indicates True and a zero value indicates `False`. In case both the methods are not implemented, all instances of the class are considered to be `True`.

<a class="anchor" id="overriding_built-in_operators"></a>
## Overriding Built-in Operators

Changing the behavior of operators is just as simple as changing the behavior of functions. You define their corresponding special methods in your class, and the operators work according to the behavior defined in these methods.

These are different from the above special methods in the sense that they need to accept another argument in the definition other than `self`, generally referred to by the name `other`. Let’s look at a few examples.

<a class="anchor" id="making_your_objects_capable_of_being_added_using_`+`"></a>
### Making Your Objects Capable of Being Added Using `+`

The special method corresponding to the `+` operator is the `__add__()` method. Adding a custom definition of `__add__()` changes the behavior of the operator. It is recommended that `__add__()` returns a new instance of the class instead of modifying the calling instance itself.

In [ ]:
class Order:
    def __init__(self, cart, customer):
        self.cart = list(cart)
        self.customer = customer

    def __add__(self, other):
        new_cart = self.cart.copy()
        new_cart.append(other)
        return Order(new_cart, self.customer)

In [ ]:
order = Order(["Apple"], "Ali")

In [ ]:
(order + 'orange').cart  # New Order instance

In [ ]:
order.cart  # Original instance unchanged

In [ ]:
order = order + 'mango'  # Changing the original instance
order.cart

Similarly, you have the `__sub__()`, `__mul__()`, and other special methods which define the behavior of `-`, `*`, and so on. These methods should return a new instance of the class as well.

<a class="anchor" id="indexing_and_slicing_your_objects_using_`[]`"></a>
### Indexing and Slicing Your Objects Using `[]`

The `[]` operator is called the indexing operator and is used in various contexts in Python such as getting the value at an index in sequences, getting the value associated with a key in dictionaries, or obtaining a part of a sequence through slicing. You can change its behavior using the `__getitem__()` special method.

In [ ]:
class Order:
    def __init__(self, cart, customer):
        self.cart = list(cart)
        self.customer = customer

    def __getitem__(self, key):
        return self.cart[key]

In [ ]:
order = Order(['banana', 'apple'], 'Ali Hejazizo')
order[0]

In [ ]:
order[-1]

In [ ]:
order[:2]

In [ ]:
o = Order([], "")

You’ll notice that above, the name of the argument to `__getitem__()` is not index but key. This is because the argument can be of mainly three forms:
- **an integer value**: in which case it is either an index or a dictionary key
- **a string value**: in which case it is a dictionary key
- **a slice object**: in which case it will slice the sequence used by the class. While there are other possibilities, these are the ones most commonly encountered.

In [ ]:
order[:]

**Note:** There is a similar `__setitem__()` special method that is used to define the behavior of `obj[x] = y`. This method takes two arguments in addition to self, generally called key and value, and can be used to change the value at key to value.

[Read More](https://realpython.com/operator-function-overloading/#the-python-data-model)

## Conclusion

In this tutorial, you learned about the Python Data Model and how the Data Model can be used to build Pythonic classes. You learned about changing the behavior of built-in functions such as `len()`, `abs()`, `str()`, `bool()`, and so on. You also learned about changing the behavior of built-in operators like `+`, `-`, `*`, `**`, and so forth.

After reading this, you can confidently create classes that make use of the best idiomatic features of Python and make your objects Pythonic!

For more information on the Data Model, and function and operator overloading, take a look at these resources:

- [Section 3.3, Special Method Names](https://docs.python.org/3/reference/datamodel.html#special-method-names) of the Data Model section in the Python documentation
- [Fluent Python](https://realpython.com/asins/1491946008/) by Luciano Ramalho
- [Python Tricks: The Book](https://realpython.com/products/python-tricks-book/)